# Databricks Health & Life Sciences
### Webinar FY25 Q4
## Prior Authorization Review with DBSQL, Variant, dbignite & redox-ehr-api

*** 

## Exploratory Analysis
***

In [0]:
DECLARE OR REPLACE VARIABLE prior_auth_bundle_uuid STRING DEFAULT "a710df7d-92b8-4e3e-99ca-b5df37808bc9";

SET VARIABLE prior_auth_bundle_uuid = :bundle_uuid;

SELECT prior_auth_bundle_uuid;

In [0]:
USE redox.hls_webinar_fy25q4;

In [0]:
SELECT 
  * 
FROM 
  fhir_bronze_variant 
WHERE 
  bundle_uuid = prior_auth_bundle_uuid;

In [0]:
SELECT 
  bundle_uuid
  ,entry.value:fullUrl::string as fullUrl
  ,entry.value:resource.resourceType::string as resourceType
  ,entry.value as resource
FROM 
  fhir_bronze_variant
  ,LATERAL variant_explode(fhir:entry) as entry
WHERE 
  bundle_uuid = prior_auth_bundle_uuid

In [0]:
WITH entry_exploded as (
  SELECT 
    bundle_uuid
    ,entry.value:fullUrl::string as fullUrl
    ,entry.value:resource.resourceType::string as resourceType
    ,entry.value as resource
  FROM 
    fhir_bronze_variant
    ,LATERAL variant_explode(fhir:entry) as entry
  WHERE 
    bundle_uuid = prior_auth_bundle_uuid
)
,pivoted_reources as (
  SELECT 
    * 
  FROM (
    SELECT
      bundle_uuid
      ,fullUrl
      ,resourceType
      ,resource
    FROM 
      entry_exploded
    )
  PIVOT (
    first(resource) FOR resourceType IN (
      'Claim'
      ,'Patient'
      ,'Practitioner'
      ,'Organization'
      ,'Coverage'
      ,'RelatedPerson'
      ,'ServiceRequest'
    )
  )
)
SELECT
  bundle_uuid
  ,collect_list(Claim)::variant as Claim
  ,collect_list(Patient)::variant as Patient
  ,collect_list(Practitioner)::variant as Practitioner
  ,collect_list(Organization)::variant as Organization
  ,collect_list(Coverage)::variant as Coverage
  ,collect_list(RelatedPerson)::variant as RelatedPerson
  ,collect_list(ServiceRequest)::variant as ServiceRequest
FROM
  pivoted_reources
GROUP BY
  bundle_uuid
;

In [0]:
DECLARE OR REPLACE VARIABLE pa_view_stmnt STRING;

SET VARIABLE pa_view_stmnt = "
  CREATE OR REPLACE VIEW prior_auth_request_bundle AS
  WITH entry_exploded as (
    SELECT 
      bundle_uuid
      ,entry.value:fullUrl::string as fullUrl
      ,entry.value:resource.resourceType::string as resourceType
      ,entry.value as resource
    FROM 
      fhir_bronze_variant
      ,LATERAL variant_explode(fhir:entry) as entry
    WHERE 
      bundle_uuid = '" || prior_auth_bundle_uuid || "'
  )
  ,pivoted_reources as (
    SELECT 
      * 
    FROM (
      SELECT
        bundle_uuid
        ,fullUrl
        ,resourceType
        ,resource
      FROM 
        entry_exploded
      )
    PIVOT (
      first(resource) FOR resourceType IN (
        'Claim'
        ,'Patient'
        ,'Practitioner'
        ,'Organization'
        ,'Coverage'
        ,'RelatedPerson'
        ,'ServiceRequest'
      )
    )
  )
  SELECT
    bundle_uuid
    ,collect_list(Claim)::variant as Claim
    ,collect_list(Patient)::variant as Patient
    ,collect_list(Practitioner)::variant as Practitioner
    ,collect_list(Organization)::variant as Organization
    ,collect_list(Coverage)::variant as Coverage
    ,collect_list(RelatedPerson)::variant as RelatedPerson
    ,collect_list(ServiceRequest)::variant as ServiceRequest
  FROM
    pivoted_reources
  GROUP BY
    bundle_uuid
  ;
";

SELECT pa_view_stmnt;

EXECUTE IMMEDIATE pa_view_stmnt;

In [0]:
SELECT 
  Claim:[0].resource.id::string as claim_id
  ,Claim:[0].resource.use::string as use
  ,Claim:[0].resource.created::TIMESTAMP as claim_created_datetime
  ,Claim:[0].resource.diagnosis[0].diagnosisCodeableConcept.coding[0].code::string as diag_code_1
  ,Claim:[0].resource.diagnosis[1].diagnosisCodeableConcept.coding[0].code::string as diag_code_2
  ,Claim:[0].resource.diagnosis[2].diagnosisCodeableConcept.coding[0].code::string as diag_code_3
  ,Claim:[0].resource.item[0].productOrService.coding[0].code::string as proc_code_1
  ,Claim:[0].resource.priority.coding[0].code::string as priority
  ,Claim:[0].resource.status::string as claim_status
  ,Claim:[0].resource.type.coding[0].code::string as claim_type
  ,ai_query(
      endpoint => 'databricks-meta-llama-3-3-70b-instruct'
      ,request => 'Lookup the ICD10 CM code descriptions for ' || Claim:[0].resource.diagnosis[0].diagnosisCodeableConcept.coding[0].code::string || ', ' || Claim:[0].resource.diagnosis[1].diagnosisCodeableConcept.coding[0].code::string || ', and ' || Claim:[0].resource.diagnosis[2].diagnosisCodeableConcept.coding[0].code::string || '. Then summarize in less than 200 characters.'
      ,returnType => 'STRING'
    ) AS diagnosis_code_summary
  ,ServiceRequest:[0].resource.code:coding[0].code::string as requested_pro_code
  ,ai_query(
    endpoint => 'databricks-meta-llama-3-3-70b-instruct'
    ,request => 'lookup HCPCS Procedure Code description for ' || ServiceRequest:[0].resource.code:coding[0].code::string
    ,returnType => 'STRING'
  ) AS requested_procedure_description
  ,*
FROM 
  prior_auth_request_bundle;

In [0]:
SELECT 
  bundle_uuid,
  struct(
    schema_of_variant(Claim) AS Claim_schema,
    schema_of_variant(Patient) AS Patient_schema,
    schema_of_variant(Organization) AS Organization_schema,
    schema_of_variant(Coverage) AS Coverage_schema,
    schema_of_variant(column5) AS column5
    -- Add more columns as needed
  ) AS struct_columns
FROM 
  prior_auth_request_bundle;


'Claim'
        ,'Patient'
        ,'Practitioner'
        ,'Organization'
        ,'Coverage'
        ,'RelatedPerson'
        ,'ServiceRequest